In [1]:
# Import the game
import gym_super_mario_bros
# Import the joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

Creo mi environment usando gym_super_mario_bros, esta libreria emula el Super Mario Bros con nes_py.
Ademas usare un entorno de actions reducido con SIMPLE_MOVEMENT

Mi idea es entrenar una AI con el primer nivel de Mario y comprobar su rendimiento posterior en niveles aleatorios.

In [2]:
# ### Codigo para el test
# env_test.seed(42)
# env_test = gym_super_mario_bros.make('SuperMarioBrosRandomStages-v0', stages=['1-2', '1-3', '1-4', '2-1'])

In [3]:
env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [4]:
env.action_space

Discrete(7)

In [5]:
# Este codigo ejecuta una accion aleatoriamente, codigo de prueba
done = True

for step in range(5000):
    if done:
        state = env.reset()
    state, reward, done, info = env.step(env.action_space.sample())
    env.render()

env.close()

c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


In [6]:
# Mi idea es crear un wrapper mas adelante para modificar mi funcion de reward y anadir recompensas por coger monedas y setas para subir el nivel
info

{'coins': 0,
 'flag_get': False,
 'life': 2,
 'score': 200,
 'stage': 1,
 'status': 'small',
 'time': 160,
 'world': 1,
 'x_pos': 722,
 'x_pos_screen': 79,
 'y_pos': 101}

In [7]:
# Con estos wrappers hare modificaciones a mi imagen para prepararlas para mi AI
from gym.wrappers import GrayScaleObservation, ResizeObservation

# Importo los dos tipos de vectorizadores para mi entorno
# Primero usare DummyVecEnv para conseguir mi primer entrenamiento estable, una vez lo obtenga, pasare a usar SubprocVecEnv para optimizarlo.
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecFrameStack
# Usare el framestack de stable baselines ya que vectoriza directamente

# Tambien importare el modelo PPO de stable baselines. De esta manera usare un algoritmo ya implementado para configurar mis rewards y mi entorno
from stable_baselines3 import PPO 

In [8]:
# Creo una funcion para inicializar mis env en DummyVecEnv
def make_env():
    # Inicializo mi env y le paso los comandos de action
    env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    # Preparo el preprocesamiento de la imagen
    env = GrayScaleObservation(env, keep_dim=True) # Keep dim en True para mantener las dimensiones
    env = ResizeObservation(env, shape=(84, 89)) # Esta resolucion es la utilizada normalmente, aunque cambia la relacion de aspecto, solo lo hace ligeramente
    return env

In [9]:
# Compruebo el formato de mi output para ver que todo este correcto
state.shape

(240, 256, 3)

In [10]:
env = DummyVecEnv([make_env for _ in range(32)]) # Con esto creare 8 env simultaneos para acelerar el proceso

env = VecFrameStack(env, 4, channels_order='last') # Numero de frames que guardare para tener contexto

In [11]:
# TODO Añadir Callback para hacer seguimiento de mi entrenamiento
from stable_baselines3.common.callbacks import BaseCallback
import os

# Esta funcion de callback esta copiada desde la documentacion de stable_baselines3
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.model.num_timesteps % self.check_freq == 0:
            model_path = os.path.join(self.save_path, f'best_model_{self.model.num_timesteps}')
            self.model.save(model_path)
            if self.verbose > 0:
                print(f"Modelo guardado en paso {self.model.num_timesteps}")

        return True

In [12]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/' # Carpeta donde guardare los tensorboard logs
callback = TrainAndLoggingCallback(check_freq=100000, save_path=CHECKPOINT_DIR)


model = PPO("CnnPolicy", env=env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.00001, n_steps=1024)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [ ]:
model.learn(total_timesteps=1_000_000, callback=callback)

In [ ]:
model.save('./models/modeloFinal')

In [13]:
model.load('./models/modeloFinal')

c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\stable_baselines3\common\save_util.py:435: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  th_object = th.load(file_

In [14]:
import time

env = DummyVecEnv([make_env])
env = VecFrameStack(env, 4, channels_order='last') 

obs = env.reset()
done = False


for step in range(5000):
    if done:
        state = env.reset()
    action, _ = model.predict(obs, deterministic=False)
    state, reward, done, info = env.step(action)
    env.render()
    time.sleep(0.01)

env.close()

c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


KeyboardInterrupt: 

In [15]:
env.close()

In [16]:
from gym.wrappers import Monitor
import os

def make_recording_env(video_folder="./videos"):
    env = make_env()  # Tu función original
    os.makedirs(video_folder, exist_ok=True)
    env = Monitor(env, video_folder, force=True)
    return env

In [19]:
model = PPO.load("./models/modeloFinal.zip")
record_env = DummyVecEnv([make_recording_env])
record_env = VecFrameStack(record_env, n_stack=4, channels_order='last')
obs = record_env.reset()

for _ in range(10000):
    action, _ = model.predict(obs, deterministic=False)
    obs, reward, done, info = record_env.step(action)
    if done:
        obs = record_env.reset()

record_env.close()

Exception ignored in: <function Monitor.__del__ at 0x00000131322E4040>
Traceback (most recent call last):
  File "c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\gym\wrappers\monitor.py", line 289, in __del__
    self.close()
  File "c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\gym\wrappers\monitor.py", line 178, in close
    super(Monitor, self).close()
  File "c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\gym\core.py", line 298, in close
    return self.env.close()
  File "c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\gym\core.py", line 298, in close
    return self.env.close()
  File "c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\gym\core.py", line 298, in close
    return self.env.close()
  [Previous line repeated 2 more times]
  File "c:\Users\Carlos\anaconda3\envs\mario_env\lib\site-packages\nes_py\nes_env.py", line 346, in close
    raise ValueError('env has already been closed.')
ValueError: env has already been cl